In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.set(rc={'ytick.labelcolor':'black','xtick.labelcolor':'black'}) 

<h1> EDA <h1>

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print('Unique values in column %s' %col,'\n', df[col].unique(),
        '\nNumber of unique values in column %s :' %col, df[col].nunique())

<h2> Visualizations <h2>

In [ ]:
for i, col in enumerate(df.columns):
    plt.figure(i)
    sns.histplot(x = col, data = df)

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.boxplot(x = col, data = df)

In [ ]:
sns.pairplot(df)

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.violinplot(x = col, data = df)

In [ ]:
information_by_age = df.groupby('age').agg(
    median_charges = ('charges', np.median), 
    mean_bmi = ('bmi', np.mean)
    ).reset_index()

In [ ]:
percent_of_smokers_age = (df.groupby('age')['smoker_yes'].sum() / (df.groupby('age')['smoker_no'].sum() + df.groupby('age')['smoker_yes'].sum())).reset_index()
percent_of_smokers_age = percent_of_smokers_age.drop(columns = 'age')
information_by_age['percent_of_smokers'] = round(percent_of_smokers_age, 2) * 100


In [ ]:
information_by_age

In [ ]:
sns.scatterplot(x = 'age', y = 'median_charges', data = information_by_age)

In [ ]:
sns.scatterplot(x = 'age', y = 'mean_bmi', data = information_by_age)

In [ ]:
sns.scatterplot(x = 'age', y = 'percent_of_smokers', data = information_by_age)

In [ ]:
information_by_region = df.groupby('region').agg(
    median_charges = ('charges', np.median), 
    mean_bmi = ('bmi', np.mean),
    mean_children = ('children', np.mean)
    ).reset_index()

In [ ]:
percent_of_smokers_region = (df.groupby('region')['smoker_yes'].sum() / (df.groupby('region')['smoker_no'].sum() + df.groupby('region')['smoker_yes'].sum())).reset_index()
percent_of_smokers_region = percent_of_smokers_region.drop(columns = 'region')
information_by_region['percent_of_smokers'] = round(percent_of_smokers_region, 2) * 100

In [ ]:
sns.barplot(x = 'region', y = 'median_charges', data = information_by_region)

In [ ]:
sns.barplot(x = 'region', y = 'mean_bmi', data = information_by_region)

In [ ]:
sns.barplot(x = 'region', y = 'mean_children', data = information_by_region)

In [ ]:
sns.barplot(x = 'region', y = 'percent_of_smokers', data = information_by_region)

<h3> Doing one-hot encoding <h3>

In [ ]:
dummies_sex = pd.get_dummies(df['sex'], prefix='sex')
dummies_smoker = pd.get_dummies(df['smoker'], prefix='smoker')
dummies_region = pd.get_dummies(df['region'], prefix='region')

In [ ]:
cols = ['sex', 'smoker', 'region']
dfs = [df, dummies_sex, dummies_smoker, dummies_region]
df = pd.concat(dfs, axis = 1)
df = df.drop(columns = cols)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

<h2>  Thoughts after performing EDA <h2>
<h3>. <h3>